In [22]:
import numpy as np
import pandas as pd

from IPython.display import clear_output

In [4]:
train = pd.read_csv('../data/train.csv', usecols=['tipodepropiedad'])
test = pd.read_csv('../data/test.csv', usecols=['tipodepropiedad'])

train_places = pd.read_csv('../data/features/ciudad_prov_lat_lng_V2_train.csv', usecols=['provincia', 'ciudad'])
test_places = pd.read_csv('../data/features/ciudad_prov_lat_lng_V2_test.csv', usecols=['provincia', 'ciudad'])

In [5]:
train = train.join(train_places)
test = test.join(test_places)

Todos los nulos están es tipodepropiedad.

# Primero se rellena el train

In [8]:
modas_ciudades_train = train.dropna().groupby('ciudad').agg(lambda x:x.value_counts().index[0])
modas_provincias_train = train.dropna().groupby('provincia').agg(lambda x:x.value_counts().index[0])

modas_ciudades_train.drop(columns=['provincia'], inplace=True)
modas_provincias_train.drop(columns=['ciudad'], inplace=True)

In [19]:
display(modas_provincias_train.head(3))
modas_provincias_train['tipodepropiedad'].value_counts()

,tipodepropiedad
provincia,
Aguascalientes,Casa
Baja California Norte,Casa
Baja California Sur,Casa


Casa           30
Apartamento     2
Name: tipodepropiedad, dtype: int64

In [20]:
display(modas_ciudades_train.head(3))
modas_ciudades_train['tipodepropiedad'].value_counts()

,tipodepropiedad
ciudad,
Abalá,Terreno
Abasolo,Casa
Abejones,Casa


Casa                  670
Terreno                71
Apartamento            31
Rancho                 27
Quinta Vacacional      19
Local Comercial        19
Villa                  16
Casa en condominio      8
Edificio                6
Terreno comercial       5
Bodega comercial        5
Huerta                  4
Casa uso de suelo       1
Nave industrial         1
Terreno industrial      1
Name: tipodepropiedad, dtype: int64

In [18]:
indice1 = train.loc[train['tipodepropiedad'].isnull()].index
indice1.size

46

In [23]:
%%time
ini = 0
fin = indice1.size
i=1
for index in indice1[ini:fin] :
    clear_output(wait=True)
    # La condición está porque hay ciudades que no registran tipos de propiedad.
    if(train.loc[index, 'ciudad'] in modas_ciudades_train.index):
        train.loc[index, 'tipodepropiedad'] = modas_ciudades_train.loc[train.loc[index, 'ciudad'], 'tipodepropiedad']
    print(i, 'de', fin - ini)
    i = i + 1

46 de 46
CPU times: user 1.04 s, sys: 38.5 ms, total: 1.08 s
Wall time: 1.04 s


In [24]:
train.isnull().sum()

tipodepropiedad    0
ciudad             0
provincia          0
dtype: int64

Ya se completó

# Ahora se rellena el test

In [25]:
modas_ciudades_test = test.dropna().groupby('ciudad').agg(lambda x:x.value_counts().index[0])
modas_provincias_test = test.dropna().groupby('provincia').agg(lambda x:x.value_counts().index[0])

modas_ciudades_test.drop(columns=['provincia'], inplace=True)
modas_provincias_test.drop(columns=['ciudad'], inplace=True)

In [26]:
display(modas_provincias_test.head(3))
modas_provincias_test['tipodepropiedad'].value_counts()

,tipodepropiedad
provincia,
Aguascalientes,Casa
Baja California Norte,Casa
Baja California Sur,Casa


Casa           30
Apartamento     2
Name: tipodepropiedad, dtype: int64

In [27]:
display(modas_ciudades_test.head(3))
modas_ciudades_test['tipodepropiedad'].value_counts()

,tipodepropiedad
ciudad,
Abasolo,Terreno
Abejones,Terreno
Acajete,Casa


Casa                             436
Terreno                           44
Apartamento                       23
Rancho                            19
Local Comercial                   16
Villa                             13
Terreno comercial                  8
Casa en condominio                 8
Quinta Vacacional                  7
Bodega comercial                   5
Inmuebles productivos urbanos      1
Edificio                           1
Name: tipodepropiedad, dtype: int64

In [29]:
indice2 = test.loc[test['tipodepropiedad'].isnull()].index
indice2.size

7

In [30]:
%%time
ini = 0
fin = indice2.size
i=1
for index in indice2[ini:fin] :
    clear_output(wait=True)
    # La condición está porque hay ciudades que no registran tipos de propiedad.
    if(test.loc[index, 'ciudad'] in modas_ciudades_test.index):
        test.loc[index, 'tipodepropiedad'] = modas_ciudades_test.loc[test.loc[index, 'ciudad'], 'tipodepropiedad']
    print(i, 'de', fin - ini)
    i = i + 1

7 de 7
CPU times: user 55.4 ms, sys: 256 µs, total: 55.7 ms
Wall time: 51.2 ms


In [31]:
test.isnull().sum()

tipodepropiedad    0
ciudad             0
provincia          0
dtype: int64

# Se pueden quitar las columnas 'ciudad' y 'provincia'

In [34]:
train.drop(columns=['ciudad', 'provincia'], inplace=True)
test.drop(columns=['ciudad', 'provincia'], inplace=True)

In [35]:
train.to_csv('../data/features/tipodepropiedad_completo_train.csv', index=False)

In [38]:
prueba = pd.read_csv('../data/features/tipodepropiedad_completo_train.csv')
prueba

,tipodepropiedad
0,Apartamento
1,Casa en condominio
2,Casa
3,Casa
4,Apartamento
...,...
239995,Casa
239996,Casa
239997,Apartamento
239998,Casa


In [39]:
test.to_csv('../data/features/tipodepropiedad_completo_test.csv', index=False)

In [40]:
prueba = pd.read_csv('../data/features/tipodepropiedad_completo_test.csv')
prueba

,tipodepropiedad
0,Casa
1,Apartamento
2,Apartamento
3,Apartamento
4,Casa
...,...
59995,Casa
59996,Casa
59997,Casa
59998,Casa


In [41]:
import category_encoders as ce

---
---
# Binary Encoding
---
---

In [51]:
# # Se juntan todos los tipos de propiedad
# vector_tipo_prop = train['tipodepropiedad']
# vector_tipo_prop = vector_tipo_prop.append(test['tipodepropiedad'])
# # Se instancia el encoder
# BI_encoder = ce.BinaryEncoder()
# # Se entrena el encoder
# BI_encoder.fit(vector_tipo_prop)

BinaryEncoder(cols=None, drop_invariant=False, handle_missing='value',
              handle_unknown='value', mapping=None, return_df=True, verbose=0)

In [55]:
# # Se pasa a categóricos.
# train = train.astype('category')
# test = test.astype('category')

In [56]:
# df_train = BI_encoder.transform(train['tipodepropiedad'])
# df_test = BI_encoder.transform(test['tipodepropiedad'])

In [58]:
# df_train.to_csv('../data/features/ftr_tipo_prop_be_train.csv', index=False)
# df_test.to_csv('../data/features/ftr_tipo_prop_be_test.csv', index=False)

---
---
# One Hot Encoding
---
---

In [61]:
# Se instancia el encoder
OH_encoder = ce.OneHotEncoder()
# Se entrena el encoder
OH_encoder.fit(vector_tipo_prop)

OneHotEncoder(cols=['tipodepropiedad'], drop_invariant=False,
              handle_missing='value', handle_unknown='value', return_df=True,
              use_cat_names=False, verbose=0)

In [62]:
df_train2 = OH_encoder.transform(train['tipodepropiedad'])
df_test2 = OH_encoder.transform(test['tipodepropiedad'])

In [63]:
df_train2.to_csv('../data/features/ftr_tipo_prop_ohe_V2_train.csv', index=False)
df_test2.to_csv('../data/features/ftr_tipo_prop_ohe_V2_test.csv', index=False)

---
---
# Label Encoding
---
---

In [64]:
# from sklearn.preprocessing import LabelEncoder

In [65]:
# # Se instancia el encoder
# LE_encoder = LabelEncoder()
# # Se entrena el encoder
# LE_encoder.fit(vector_tipo_prop)

LabelEncoder()

In [66]:
# df_train3 = LE_encoder.transform(train['tipodepropiedad'])
# df_test3 = LE_encoder.transform(test['tipodepropiedad'])

In [71]:
# df_train3 = pd.DataFrame(data=df_train3, columns=['tipodepropiedad_LE'])
# df_test3 = pd.DataFrame(data=df_test3, columns=['tipodepropiedad_LE'])

In [74]:
# df_train3.to_csv('../data/features/ftr_tipo_prop_le_train.csv', index=False)
# df_test3.to_csv('../data/features/ftr_tipo_prop_le_test.csv', index=False)